<a href="https://colab.research.google.com/github/tzuyu1224/program/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW5 RAG

## 1. 請完成以下程式 (50%)

以下程式碼有許多不完整的地方，請依照原本的程式框架，補齊所有未完成的部分，請確保所有程式碼能完整執行並得到合理的回答結果。

In [ ]:
# ✅ STEP 1: 安裝需要的套件

!pip install -q sentence-transformers faiss-cpu requests

In [ ]:
# ✅ STEP 2: 載入必要模組

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import requests
import pandas as pd

In [ ]:
### 非台大學生請執行以下程式碼獲取語料 ###

!gdown 1AF2g2WTtXQwb02S5EAD7aL48c9BkV1ed

### 非台大學生請執行以上程式碼獲取語料 ###

In [ ]:
# ✅ STEP 3: 準備你的語料

docs_df = ("dcard-top100.csv")

documents = docs_df["content"].tolist()
docs_df

Downloading...
From: https://drive.google.com/uc?id=1AF2g2WTtXQwb02S5EAD7aL48c9BkV1ed
To: /content/dcard-top100.csv
100% 186k/186k [00:00<00:00, 23.5MB/s]


,ID,title,content
0,235443715,三更 21歲這年我差點被活活燒死,部分回應在B117 \r\n謝謝各位的留言，我都有看完\r\n好的不好的，我都接受謝謝大家🙇...
1,235442609,超狂學經歷！195公分帥家教徵學生,https://i.imgur.com/REIEzSd.jpg\r\n\r\n身高195公分...
2,235441998,我的模特界時間管理大師前男友,看過這麼多在Dcard、PTT上的感情渣事和創作文\r\n從沒想過如此荒謬像八點檔的事情居然...
3,235441259,豆皮加爆,剛剛吃小火鍋，跟店員說不要金針菇（怕卡牙縫），於是店員幫我換其他配料..…\r\n\r\n沒...
4,235442693,這樣女生該追嗎,已經約好見面，到了當天晚上七點半才回，我是被耍了嗎 \r\n如下圖\r\n\r\n\r\nh...
...,...,...,...
95,235444562,主題：💟愛情塔羅💟 他要的感情關係vs 我要的感情關係為何？兩個人的近期發展（曖昧/交往中/...,🤗感謝我們之間的連結帶領你來到這裡！ \r\n\r\n主題：💟愛情塔羅💟 他要的感情關係vs...
96,235441027,#分享 老花古董包 LV 方盒子｜Celine 化妝箱,近期購入的兩款古董包 可能因為是新歡 所以都讓我愛不釋手(๑•̀ •́)و✧\r\n不過正是...
97,235447320,在公車上遇到的超尷尬事件,剛剛在公車上有一個阿伯下車前跟我說：妹妹你很有愛心欸 讓位置給別人 不像有些人都在裝睡\r\...
98,235440749,#大眾占卜 🥺。他現在對我的想法是?他的下一步是什麼?我能為這段關係做什麼樣的努力❣,大家好這裡是金魚🙏\r\n這次的占卜適合斷聯、曖昧、復合的族群\r\n\r\n再上圖之前先置...


In [ ]:
# ✅ STEP 4: 建立 embedding 模型與 FAISS 索引

embedding_model = SentenceTransformer("shibing624/text2vec-base-multilingual")

# 轉換為向量並正規化（便於 cosine 相似度）
doc_embeddings = embedding_model.encode(documents, normalize_embeddings=True)

# 建立向量索引
index = faiss.IndexFlatIP(doc_embeddings.shape[1])
index.add(doc_embeddings)

In [ ]:
#### 作業完成後請刪除此 code chunk 後再上傳作業 ####

# 你的 api token

HUGGINGFACE_TOKEN = ""

#### 作業完成後請刪除此 code chunk 後再上傳作業 ####

In [ ]:
# 🤖 STEP 5: 呼叫 HuggingFace 模型 API

import requests

API_URL = "https://api-inference.huggingface.co/models/ClueAI/ChatYuan-large-v2"

headers = {"Authorization": f"Bearer {HUGGINGFACE_TOKEN}"}

# 請完成 prompt
def generate_answer(query, context_docs):
    context = "\n".join(f"- {doc[:300]}" for doc in context_docs)  # 避免太長
    prompt = f"""你是一個，請根據以下資料回答問題，若沒有相關資訊請回答「找不到答案」。

資料：{}

問題：{}

請用一到兩句話回答。
"""

    payload = {"inputs": prompt}
    response = requests.post(API_URL, headers=headers, json=payload, timeout=60)

    try:
        data = response.json()
        if isinstance(data, list) and "generated_text" in data[0]:
            return data[0]["generated_text"]
        elif isinstance(data, dict) and "generated_text" in data:
            return data["generated_text"]
        else:
            return f"⚠️ 無法解析模型回應：{data}"
    except Exception as e:
        return f"❌ 發生錯誤：{e}"



In [ ]:
def search(query, top_k=3):

  query_vec = embedding_model.encode([query], normalize_embeddings=True)
  scores, indices = index.search(query_vec, top_k)
  matched_docs = [documents[i] for i in indices[0]]

  return matched_docs


In [ ]:
# ✅ STEP 6: 使用者輸入問題，找出最相關的內容

query = "在哪裡可以看到很多草莓？"

relevant_docs = search(query, top_k=3)
answer = generate_answer(query, relevant_docs)

print("🔎 相關段落：\n", "\n---\n".join(relevant_docs)[:100])
print("\n💬 AI 回答：\n", answer)

🔎 相關段落：
 🔅來來牛奶草莓園🍓🍓
一轉進水尾坪這裡就可以看到好多草莓園
但是這次很堅持的是要去來來牛奶草莓園
主要的是看重這家草莓園還有提供草莓斗笠
可以讓你一邊戴著可愛斗笠一邊採收草莓
好像自己就是個

💬 AI 回答：
 答案： 水尾坪這裡就可以看到好多草莓。


## 2. 請回答以下問題 (50%)

請嘗試輸入至少 3 個問題給你的問答系統，並觀察模型回答的準確性與表現：

- 你問了哪些問題？

- 模型是否成功從語料中找到與問題相關的內容？

- 回答是否合理？是否有亂編的情況？

- 若問題無法回答，模型是否能正確回應「找不到答案」？

請根據你的觀察，簡述模型整體表現的優缺點。
